In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/project 1

/content/drive/MyDrive/project 1


In [3]:
%pwd

'/content/drive/MyDrive/project 1'

In [4]:
%pip install import-ipynb
%pip install ptflops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.0 MB/s eta 0:00:00


### Task 1

#### 1. Basic Concepts
1. The purpose of using dataset distillation in this paper is to reduce the training costs while maintaining the high performance on various machine learning tasks. The authors introduce Dataset Distillation with Attention Maching (DataDAM) to condense large datasets into smaller synthetic dataset that retain the critical information, allowing models trained on the synthetic set to achieve similar accuracy as those trained on the full dataset.
2.  The advantages are: (page 2)
- Efficient end-to-end dataset distillation: This highlights the ability of DataDAM to closely approximate the distribution of the real dataset while keeping **computational costs low**.
- Improved accuracy and scalability: DataDAM demonstrate the performance across multiple benchmark dataset and reduces the training costs by up to 100x, while also allowing for cross-architecture generation. This makes it more scalable and flexible for real-world application.
- Enhancement of downstream application: DATADAM's distilled data improves memory efficiency in continual learning tasks and accelerates neural architectures search(NAS) by providing a more representative proxy dataset, enabling a faster and more efficient learning process.  
3. The novelty includes: (page 2)
- Multiple Randomly Initialized DNNs: DataDAM uses multiple randomly initialized deep neural networks to extract meaningful representations from both real and synthetic datasets, which is different from methods that rely on pre-trained models
- Spatial attention matching (SAM): The SAM module align the most discriminative feature maps from real and synthetic datasets, reducing the gap between the dataset.
- Last-Layer Feature Alignment: It reduces disparities in the last-layer feature distributions between the real and synthetic datasets by using a complementary loss as a regularizer, ensuring high-level abstract representations are similar.
- Bias-Free Synthetic Data: The synthetic data generated by DataDAM does not introduce any bias, which is a significant improvement over prior methods, ensuring better generalization and performance.
4. The methodology of DataDAM is centered on efficiently distilling datasets through attention matching: (page 4)
- Initialization of Synthetic Dataset: The process starts by initializing a synthetic dataset, which can be done through random noise or by sampling real data.
- Feature Extraction: Real and synthetic datasets are passed through randomly initialized deep neural networks, and features are extracted at multiple layers.
- Spatial Attention Matching (SAM): Attention maps are computed for each layer, excluding the final layer. These attention maps focus on the most discriminative regions of the input image.
- Loss Functions:
    - SAM Loss (LSAM): This loss minimizes the distance between attention maps of real and synthetic datasets across layers.
    - Maximum Mean Discrepancy Loss (MMD): This complementary loss aligns the last-layer feature distributions of the two datasets, ensuring the high-level abstract information is captured.
- Optimization: The synthetic dataset is optimized using a combination of the SAM loss and LMMD loss to minimize the difference between real and synthetic data.
5. (page 8)
- Continual Learning: DataDAM’s ability to condense datasets efficiently makes it highly useful in continual learning scenarios, where a model must learn incrementally while preventing catastrophic forgetting. By using the distilled datasets as a replay buffer, DataDAM can significantly improve memory efficiency and performance in incremental learning tasks.
- Neural Architecture Search (NAS): The synthetic datasets generated by DataDAM can serve as proxies in NAS tasks, allowing faster evaluation of model architectures. This leads to a significant reduction in computational costs and time during the model search process, making NAS more feasible in real-world applications.

#### 2. Data Distillation Learning - MHIST
##### a) Train model with origianl dataset


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import SGD
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import CosineAnnealingLR

In [6]:
import import_ipynb
from utils import get_network, get_time

In [ ]:
train_folder = 'mhist_dataset/train'
test_folder = 'mhist_dataset/test'

transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = datasets.ImageFolder(root=train_folder, transform=transform)
test_dataset = datasets.ImageFolder(root=test_folder, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
model = get_network(model='ConvNetD7', channel=3, num_classes=2, im_size=(224, 224))
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = CosineAnnealingLR(optimizer, T_max=20)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def training_model(model, optimizer, scheduler, criterion, train_loader):
    torch.cuda.empty_cache()
    for epoch in range(20):
        model.train()

        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            labels = labels.long()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            torch.cuda.empty_cache()
        scheduler.step()

        print(f"Epoch [{epoch+1}/20], Loss: {running_loss / len(train_loader.dataset)}")


In [ ]:
training_model(model, optimizer, scheduler, criterion, train_loader)

Epoch [1/20], Loss: 0.5949159725233056
Epoch [2/20], Loss: 0.528848231743122
Epoch [3/20], Loss: 0.4651013902686108
Epoch [4/20], Loss: 0.4230009388238534
Epoch [5/20], Loss: 0.4042419616244305
Epoch [6/20], Loss: 0.34856274581503593
Epoch [7/20], Loss: 0.2842565602132644
Epoch [8/20], Loss: 0.2573209328624024
Epoch [9/20], Loss: 0.1899680537464975
Epoch [10/20], Loss: 0.14501041364395756
Epoch [11/20], Loss: 0.09171520350993365
Epoch [12/20], Loss: 0.050883273871979495
Epoch [13/20], Loss: 0.027555900538104704
Epoch [14/20], Loss: 0.015726765303647725
Epoch [15/20], Loss: 0.009714267691490294
Epoch [16/20], Loss: 0.008019728065607534
Epoch [17/20], Loss: 0.007051664652923743
Epoch [18/20], Loss: 0.006584131865194817
Epoch [19/20], Loss: 0.0063520952536799445
Epoch [20/20], Loss: 0.006245654145491192


In [ ]:
model_path = 'models/mhist_original.pth'
torch.save(model.state_dict(), model_path)

In [ ]:
model_path = 'models/mhist_original.pth'
model = get_network(model='ConvNetD7', channel=3, num_classes=2, im_size=(224, 224))
model.load_state_dict(torch.load(model_path))


<ipython-input-12-068f2300fe71>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def evaluate_model(model, test_loader):
    model.to(device)
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
      for inputs, labels in test_loader:
          inputs, labels = inputs.to(device), labels.to(device)
          labels = labels.long()
          outputs = model(inputs)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

In [ ]:
evaluate_model(model, test_loader)

Test Accuracy: 81.06%


In [ ]:
from ptflops import get_model_complexity_info

macs, params = get_model_complexity_info(model, (3, 224, 224), as_strings=True, print_per_layer_stat=True)
print(f"MACs: {macs}, Parameters: {params}")

ConvNet(
  891.14 k, 100.000% Params, 2.68 GMac, 99.365% MACs, 
  (features): Sequential(
    890.88 k, 99.971% Params, 2.68 GMac, 99.365% MACs, 
    (0): Conv2d(3.58 k, 0.402% Params, 179.83 MMac, 6.664% MACs, 3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): GroupNorm(256, 0.029% Params, 12.85 MMac, 0.476% MACs, 128, 128, eps=1e-05, affine=True)
    (2): ReLU(0, 0.000% Params, 6.42 MMac, 0.238% MACs, inplace=True)
    (3): AvgPool2d(0, 0.000% Params, 6.42 MMac, 0.238% MACs, kernel_size=2, stride=2, padding=0)
    (4): Conv2d(147.58 k, 16.561% Params, 1.85 GMac, 68.604% MACs, 128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): GroupNorm(256, 0.029% Params, 3.21 MMac, 0.119% MACs, 128, 128, eps=1e-05, affine=True)
    (6): ReLU(0, 0.000% Params, 1.61 MMac, 0.060% MACs, inplace=True)
    (7): AvgPool2d(0, 0.000% Params, 1.61 MMac, 0.060% MACs, kernel_size=2, stride=2, padding=0)
    (8): Conv2d(147.58 k, 16.561% Params, 462.82 MMac, 17.151% MACs, 128,

In [ ]:
total_flops = 0

for inputs, labels in test_loader:
    total_flops += get_model_complexity_info(model, (3, 224, 224), as_strings=False)[0]

print(f"Total FLOPs for the test dataset: {total_flops}")

ConvNet(
  891.14 k, 100.000% Params, 2.68 GMac, 99.365% MACs, 
  (features): Sequential(
    890.88 k, 99.971% Params, 2.68 GMac, 99.365% MACs, 
    (0): Conv2d(3.58 k, 0.402% Params, 179.83 MMac, 6.664% MACs, 3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): GroupNorm(256, 0.029% Params, 12.85 MMac, 0.476% MACs, 128, 128, eps=1e-05, affine=True)
    (2): ReLU(0, 0.000% Params, 6.42 MMac, 0.238% MACs, inplace=True)
    (3): AvgPool2d(0, 0.000% Params, 6.42 MMac, 0.238% MACs, kernel_size=2, stride=2, padding=0)
    (4): Conv2d(147.58 k, 16.561% Params, 1.85 GMac, 68.604% MACs, 128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): GroupNorm(256, 0.029% Params, 3.21 MMac, 0.119% MACs, 128, 128, eps=1e-05, affine=True)
    (6): ReLU(0, 0.000% Params, 1.61 MMac, 0.060% MACs, inplace=True)
    (7): AvgPool2d(0, 0.000% Params, 1.61 MMac, 0.060% MACs, kernel_size=2, stride=2, padding=0)
    (8): Conv2d(147.58 k, 16.561% Params, 462.82 MMac, 17.151% MACs, 128,

##### b) Learn the synthetic dataset with attention matching algorithm

In [ ]:
# real dataset loader
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
real_dataset = datasets.ImageFolder(root='mhist_dataset/train', transform=transform)
real_loader = torch.utils.data.DataLoader(real_dataset, batch_size=128, shuffle=True)

In [ ]:
import torch
import random
from torch import optim

num_classes = 2
num_images_per_class = 50
img_size = (3, 224, 224)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def generate_synthetic_dataset(real_dataset, num_classes=2, images_per_class=50, K=200, eta_s=0.1,
                               zeta_s=1, eta_theta=0.01, zeta_theta=50, minibatch_size=128):
    synthetic_images = []
    synthetic_labels = []

    for class_id in range(num_classes):

        class_indices = [i for i, (_, label) in enumerate(real_dataset) if label == class_id]
        sampled_indices = random.sample(class_indices, images_per_class)

        for i in sampled_indices:
            img_real = real_dataset[i][0].to(device)

            synthetic_versions = []
            for _ in range(K):
                synthetic_image = img_real.clone().detach().requires_grad_(True)
                synthetic_versions.append(synthetic_image)

            for synthetic_image in synthetic_versions:
                optimizer_condensed = optim.SGD([synthetic_image], lr=eta_s)

                for _ in range(zeta_s):
                    optimizer_condensed.zero_grad()
                    loss = torch.nn.functional.mse_loss(synthetic_image, img_real)
                    loss.backward()
                    optimizer_condensed.step()

            final_synthetic_image = torch.mean(torch.stack(synthetic_versions), dim=0).detach()

            synthetic_images.append(final_synthetic_image)
            synthetic_labels.append(class_id)

    img_syn = torch.stack(synthetic_images)
    labels_syn = torch.tensor(synthetic_labels, device=device)

    return img_syn, labels_syn


In [ ]:
#step 2: optimizer
img_syn, labels_syn = generate_synthetic_dataset(real_dataset, 2)

img_syn = torch.nn.Parameter(img_syn)
optimizer_img = optim.SGD([img_syn], lr=0.1)  # lr is eta_s

In [ ]:
# step 3: set up model - ConvNET - 7 in this case
# freeze the model's weights so that only the synthetic dataset is updated

model_path = 'models/mhist_original.pth'
net = get_network(model='ConvNetD7', channel=3, num_classes=2, im_size=(224, 224))
net.load_state_dict(torch.load(model_path, map_location=device))
net.to(device)
net.train()

for param in list(net.parameters()):
    param.requires_grad = False

<ipython-input-14-2e2ec9446353>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(model_path, map_location=device))


In [ ]:
# step 5: hook
# Using hook to extract the activations from the layers (attention maps) to compare the attention maps from real to synthetic
activations = {}


def get_activation(name):
    def hook_func(m, inp, op):
        activations[name] = op.detach()

    return hook_func


''' Defining the Refresh Function to store Activations and reset Collection '''


def refresh_activations(activations):
    model_set_activations = []  # Jagged Tensor Creation
    for i in activations.keys():
        model_set_activations.append(activations[i])
    activations = {}
    return activations, model_set_activations


def delete_hooks(hooks):
    for i in hooks:
        i.remove()
    return


def attach_hooks(net):
    hooks = []
    for name, module in net.named_modules():
        if isinstance(module, nn.ReLU):
            hooks.append(module.register_forward_hook(get_activation(name)))
    return hooks


In [ ]:
# step 6: Attention Matching Map
def get_attention(feature_set, param=0, exp=4, norm='l2'):
    if param == 0:
        attention_map = torch.sum(torch.abs(feature_set), dim=1)

    elif param == 1:
        attention_map = torch.sum(torch.abs(feature_set) ** exp, dim=1)

    elif param == 2:
        attention_map = torch.max(torch.abs(feature_set) ** exp, dim=1)

    if norm == 'l2':
        # Dimension: [B x (H*W)] -- Vectorized
        vectorized_attention_map = attention_map.view(feature_set.size(0), -1)
        normalized_attention_maps = F.normalize(vectorized_attention_map, p=2, dim=1)

    return normalized_attention_maps

In [ ]:
# step 7: error function
def error(real, syn, err_type="MSE"):
    if err_type == "MSE":
        err = torch.sum((torch.mean(real, dim=0) - torch.mean(syn, dim=0)) ** 2)
    elif err_type == "MAE":
        err = torch.sum(torch.abs(torch.mean(real, dim=0) - torch.mean(syn, dim=0)))
    elif err_type == "MSE_B":
        err = torch.sum(
            (torch.mean(real.reshape(2, -1).cpu(), dim=1) - torch.mean(syn.reshape(2, -1).cpu(), dim=1)) ** 2)
    else:
        raise ValueError("Invalid error type")
    return err

In [ ]:
# step 8: training loop
def train_dataset(img_syn, labels_syn, activations={}):
    num_iterations = 10
    learning_rate_model = 0.01

    losses = []
    for iteration in range(num_iterations):
        print(f"Iteration {iteration + 1}/{num_iterations}")

        images_syn_all = []
        labels_syn_all = []
        images_real_all = []
        labels_real_all = []

        for c in range(num_classes):
            img_real, label_real = next(iter(real_loader))
            img_real = img_real.to(device)
            img_syn_per_class = img_syn[c * num_images_per_class:(c + 1) * num_images_per_class].to(device)
            label_syn_per_class = labels_syn[c * num_images_per_class:(c + 1) * num_images_per_class]

            images_real_all.append(img_real)
            labels_real_all.append(label_real)
            images_syn_all.append(img_syn_per_class)
            labels_syn_all.append(label_syn_per_class)

        images_real_all = torch.cat(images_real_all, dim=0)
        labels_real_all = torch.cat(labels_real_all, dim=0)
        images_syn_all = torch.cat(images_syn_all, dim=0)
        labels_syn_all = torch.cat(labels_syn_all, dim=0)

        net.train()
        hooks = attach_hooks(net)

        output_real = net(images_real_all)[0]
        activations, original_model_set_activations = refresh_activations(activations)

        output_syn = net(images_syn_all)[0]
        activations, syn_model_set_activations = refresh_activations(activations)
        delete_hooks(hooks)

        length_of_network = len(original_model_set_activations)

        loss = torch.tensor(0.0)
        mid_loss = 0
        out_loss = 0
        loss_avg = 0

        for layer in range(length_of_network - 1):
            real_attention = get_attention(original_model_set_activations[layer], param=1, exp=1, norm='l2')
            syn_attention = get_attention(syn_model_set_activations[layer], param=1, exp=1, norm='l2')

            tl = 100 * error(real_attention, syn_attention, err_type="MSE_B")
            loss += tl
            mid_loss += tl.item()

        output_loss = 100 * 0.01 * error(output_real, output_syn, err_type="MSE_B")
        loss += output_loss
        out_loss += output_loss.item()

        optimizer_img.zero_grad()
        loss.backward()
        optimizer_img.step()
        loss_avg += loss.item()
        torch.cuda.empty_cache()

        loss_avg /= (num_classes)
        out_loss /= (num_classes)
        mid_loss /= (num_classes)
        losses.append((loss_avg, out_loss, mid_loss))
        if iteration % 10 == 0:
            print('%s iter = %05d, loss = %.4f' % (get_time(), iteration, loss_avg))

    print("training completed.")
    return img_syn, labels_syn, losses


In [ ]:
def save_results(img_syn, labels_syn, losses, noise_type):
    save_path = f"mhist_result/{noise_type}_synthetic_dataset.pt"

    torch.save({'images': img_syn, 'labels': labels_syn}, save_path)

    print(f"Synthetic dataset saved to {save_path}")

    loss_log_path = f"mhist_result/{noise_type}_training_losses.txt"
    with open(loss_log_path, "w") as f:
        for epoch, loss in enumerate(losses):
            f.write(f"Iteration {epoch}: Loss = {loss}\n")

    print(f"Training losses saved to {loss_log_path}")

In [ ]:
img_syn, labels_syn, losses = train_dataset(img_syn, labels_syn)
save_results(img_syn, labels_syn, losses, '')

Iteration 1/10
[2024-10-26 01:25:31] iter = 00000, loss = 0.0786
Iteration 2/10
Iteration 3/10
Iteration 4/10
Iteration 5/10
Iteration 6/10
Iteration 7/10
Iteration 8/10
Iteration 9/10
Iteration 10/10
training completed.
Synthetic dataset saved to mhist_result/_synthetic_dataset.pt
Training losses saved to mhist_result/_training_losses.txt


##### 2c) Visualize the condensed images per class

In [7]:
import torch
import os
import matplotlib.pyplot as plt

def display_image_grid(img_syn, labels_syn, noise_type):
    # Create directory to save grid images
    save_dir = f"mhist_result/{noise_type}_grid"
    os.makedirs(save_dir, exist_ok=True)

    num_samples_per_class = 50

    class_dict = {}

    for img, label in zip(img_syn, labels_syn):
        label = label.item()
        if label not in class_dict:
            class_dict[label] = []
        class_dict[label].append(img)

    for class_label, images in class_dict.items():
        selected_images = images[:num_samples_per_class]

        fig, axs = plt.subplots(5, 10, figsize=(15, 7))
        fig.suptitle(f"Class {class_label} - {noise_type}", fontsize=16)
        fig.patch.set_facecolor('white')

        # Display each image
        for idx, img in enumerate(selected_images):
            img_np = img.permute(1, 2, 0).detach().numpy()
            axs[idx // 10, idx % 10].imshow(img_np)
            axs[idx // 10, idx % 10].axis('off')
            axs[idx // 10, idx % 10].set_facecolor('white')

        # Display the grid of images
        plt.show()

        # Save the grid of images as a single image file for report
        grid_save_path = os.path.join(save_dir, f"class_{class_label}_grid.png")
        fig.savefig(grid_save_path, facecolor='white')  # Save grid with white background
        plt.close()

    print(f"Image grids saved to {save_dir}")

# Call the function with your loaded data

In [8]:
img_syn_loaded = torch.load('mhist_result/_synthetic_dataset.pt')
img_syn = img_syn_loaded['images']
labels_syn = img_syn_loaded['labels']

display_image_grid(img_syn, labels_syn, noise_type='original')

Output hidden; open in https://colab.research.google.com to view.

#### random noise

In [ ]:
# set up the synthetic dataset
import random

num_classes = 2
num_images_per_class = 50
img_size = (3, 224, 224)

def generate_synthetic_dataset_with_noise(real_dataset, num_classes, images_per_class=50, K=200, eta_s=0.01,
                                          zeta_s=1, eta_theta=0.01, zeta_theta=50, minibatch_size=128, noise_std=0.8):
    synthetic_images = []
    synthetic_labels = []

    for class_id in range(num_classes):
        # Randomly sample 50 images from each class
        class_indices = [i for i, (_, label) in enumerate(real_dataset) if label == class_id]
        sampled_indices = random.sample(class_indices, images_per_class)

        for i in sampled_indices:
            img_real = real_dataset[i][0].to(device)

            noisy_versions = []
            for k_init in range(K):
                noise = torch.rand(size=img_real.shape, requires_grad=True).to(device)
                noisy_image = torch.clamp(img_real + noise, 0, 1).clone().detach().requires_grad_(True)
                noisy_versions.append(noisy_image)

            for k_init in range(K):
                noisy_image = noisy_versions[k_init]
                optimizer_condensed = optim.SGD([noisy_image], lr=eta_s)

                for step in range(zeta_s):
                    optimizer_condensed.zero_grad()
                    loss = torch.nn.functional.mse_loss(noisy_image, img_real)
                    loss.backward()
                    optimizer_condensed.step()

            selected_synthetic_image = torch.mean(torch.stack(noisy_versions), dim=0).detach()

            synthetic_images.append(selected_synthetic_image)
            synthetic_labels.append(real_dataset[i][1])

            selected_synthetic_image = noisy_versions[0].detach()
            synthetic_images.append(selected_synthetic_image)
            synthetic_labels.append(real_dataset[i][1])

    img_syn = torch.stack(synthetic_images)
    labels_syn = torch.tensor(synthetic_labels, device=device)

    return img_syn, labels_syn

In [ ]:
img_syn, labels_syn = generate_synthetic_dataset_with_noise(real_dataset, num_classes)

img_syn = torch.nn.Parameter(img_syn)
optimizer_img = optim.SGD([img_syn], lr=0.1)  # lr is eta_s

In [ ]:
img_syn, labels_syn, losses = train_dataset(img_syn, labels_syn)
save_results(img_syn, labels_syn, losses, 'Random')

Iteration 1/10
[2024-10-25 22:45:40] iter = 00000, loss = 9.4582
Iteration 2/10
Iteration 3/10
Iteration 4/10
Iteration 5/10
Iteration 6/10
Iteration 7/10
Iteration 8/10
Iteration 9/10
Iteration 10/10
training completed.
Synthetic dataset saved to mhist_result/Random_synthetic_dataset.pt
Training losses saved to mhist_result/Random_training_losses.txt


In [9]:
img_syn_loaded = torch.load('mhist_result/Random_synthetic_dataset.pt')
img_syn = img_syn_loaded['images']
labels_syn = img_syn_loaded['labels']

display_image_grid(img_syn, labels_syn, noise_type='Random')

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 2d
def generate_synthetic_dataset_with_noise(real_dataset, num_classes, images_per_class=50, K=200, eta_s=0.01,
                                          zeta_s=1, eta_theta=0.01, zeta_theta=50, minibatch_size=128, noise_std=0.8):
    synthetic_images = []
    synthetic_labels = []

    for class_id in range(num_classes):
        # Randomly sample 50 images from each class
        class_indices = [i for i, (_, label) in enumerate(real_dataset) if label == class_id]
        sampled_indices = random.sample(class_indices, images_per_class)

        for i in sampled_indices:
            img_real = real_dataset[i][0].to(device)

            noisy_versions = []
            for k_init in range(K):
                noise = torch.normal(mean=0, std=noise_std, size=img_real.shape, requires_grad=True).to(device)
                noisy_image = torch.clamp(img_real + noise, 0, 1).clone().detach().requires_grad_(True)
                noisy_versions.append(noisy_image)

            for k_init in range(K):
                noisy_image = noisy_versions[k_init]
                optimizer_condensed = optim.SGD([noisy_image], lr=eta_s)

                for step in range(zeta_s):
                    optimizer_condensed.zero_grad()
                    loss = torch.nn.functional.mse_loss(noisy_image, img_real)
                    loss.backward()
                    optimizer_condensed.step()

            selected_synthetic_image = torch.mean(torch.stack(noisy_versions), dim=0).detach()

            synthetic_images.append(selected_synthetic_image)
            synthetic_labels.append(real_dataset[i][1])

    img_syn = torch.stack(synthetic_images)
    labels_syn = torch.tensor(synthetic_labels, device=device)

    return img_syn, labels_syn

img_syn, labels_syn = generate_synthetic_dataset_with_noise(real_dataset, num_classes)
img_syn = torch.nn.Parameter(img_syn)
optimizer_img = optim.SGD([img_syn], lr=0.1)  # lr is eta_s

img_syn, labels_syn, losses = train_dataset(img_syn, labels_syn)
save_results(img_syn, labels_syn, losses, 'Gaussian')

Iteration 1/10
[2024-10-25 21:59:58] iter = 00000, loss = 37.9859
Iteration 2/10
Iteration 3/10
Iteration 4/10
Iteration 5/10
Iteration 6/10
Iteration 7/10
Iteration 8/10
Iteration 9/10
Iteration 10/10
training completed.
Synthetic dataset saved to mhist_result/Gaussian_synthetic_dataset.pt
Training losses saved to mhist_result/Gaussian_training_losses.txt


In [10]:
img_syn_loaded = torch.load('mhist_result/Gaussian_synthetic_dataset.pt')
img_syn = img_syn_loaded['images']
labels_syn = img_syn_loaded['labels']

display_image_grid(img_syn, labels_syn, noise_type='Gaussian')

Output hidden; open in https://colab.research.google.com to view.

##### 2e) Train the model with condensed dataset

In [ ]:
random_model = get_network(model='ConvNetD7', channel=3, num_classes=2, im_size=(224, 224))
random_optimizer = SGD(random_model.parameters(), lr=0.01, momentum=0.9)

gaussian_model = get_network(model='ConvNetD7', channel=3, num_classes=2, im_size=(224, 224))
gaussian_optimizer = SGD(gaussian_model.parameters(), lr=0.01, momentum=0.9)

random_scheduler = CosineAnnealingLR(random_optimizer, T_max=20)
gaussian_scheduler = CosineAnnealingLR(gaussian_optimizer, T_max=20)

random_criterion = torch.nn.CrossEntropyLoss()
gaussian_criterion = torch.nn.CrossEntropyLoss()

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

img_syn_loaded = torch.load('mhist_result/Random_synthetic_dataset.pt')
img_syn = img_syn_loaded['images']
labels_syn = img_syn_loaded['labels']

random_synthetic_dataset = TensorDataset(img_syn, labels_syn)

random_train_loader = DataLoader(random_synthetic_dataset, batch_size=32, shuffle=True)

training_model(random_model, random_optimizer, random_scheduler,
               random_criterion, random_train_loader)

<ipython-input-52-42573f96207b>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  img_syn_loaded = torch.load('mhist_result/Random_synthetic_dataset.pt')


Epoch [1/20], Loss: 0.6997072148323059
Epoch [2/20], Loss: 0.622557303905487
Epoch [3/20], Loss: 0.512061460018158
Epoch [4/20], Loss: 0.3991895890235901
Epoch [5/20], Loss: 0.2558365648984909
Epoch [6/20], Loss: 0.1334417223930359
Epoch [7/20], Loss: 0.060247775465250016
Epoch [8/20], Loss: 0.02868585579097271
Epoch [9/20], Loss: 0.017183593884110452
Epoch [10/20], Loss: 0.009771309494972229
Epoch [11/20], Loss: 0.007616597861051559
Epoch [12/20], Loss: 0.006195605974644422
Epoch [13/20], Loss: 0.00548469003289938
Epoch [14/20], Loss: 0.005047740451991558
Epoch [15/20], Loss: 0.004782806783914566
Epoch [16/20], Loss: 0.004614790007472038
Epoch [17/20], Loss: 0.004508841577917338
Epoch [18/20], Loss: 0.004447358436882496
Epoch [19/20], Loss: 0.004413905907422304
Epoch [20/20], Loss: 0.004400953482836485


In [ ]:

img_syn_loaded = torch.load('mhist_result/Gaussian_synthetic_dataset.pt')
img_syn = img_syn_loaded['images']
labels_syn = img_syn_loaded['labels']

gaussian_synthetic_dataset = TensorDataset(img_syn, labels_syn)

gaussian_train_loader = DataLoader(gaussian_synthetic_dataset, batch_size=32, shuffle=True)

training_model(gaussian_model, gaussian_optimizer, gaussian_scheduler,
               gaussian_criterion, gaussian_train_loader)

<ipython-input-53-7c64865d9e6d>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  img_syn_loaded = torch.load('mhist_result/Gaussian_synthetic_dataset.pt')


Epoch [1/20], Loss: 0.7211712956428528
Epoch [2/20], Loss: 0.6415694522857666
Epoch [3/20], Loss: 0.6026229524612426
Epoch [4/20], Loss: 0.5594196259975434
Epoch [5/20], Loss: 0.4966701078414917
Epoch [6/20], Loss: 0.4501708221435547
Epoch [7/20], Loss: 0.4054525399208069
Epoch [8/20], Loss: 0.36042810559272764
Epoch [9/20], Loss: 0.26134994983673093
Epoch [10/20], Loss: 0.2680172801017761
Epoch [11/20], Loss: 0.20254085302352906
Epoch [12/20], Loss: 0.2090478616952896
Epoch [13/20], Loss: 0.15104658246040345
Epoch [14/20], Loss: 0.1867198419570923
Epoch [15/20], Loss: 0.10710520148277283
Epoch [16/20], Loss: 0.09391091197729111
Epoch [17/20], Loss: 0.09327208727598191
Epoch [18/20], Loss: 0.07799613371491432
Epoch [19/20], Loss: 0.06969380050897599
Epoch [20/20], Loss: 0.0666884446144104


In [ ]:
test_folder = 'mhist_dataset/test'

transform = transforms.Compose([
    transforms.ToTensor()
])

test_dataset = datasets.ImageFolder(root=test_folder, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
evaluate_model(random_model, test_loader)

Test Accuracy: 58.34%


In [ ]:
evaluate_model(gaussian_model, test_loader)

Test Accuracy: 64.07%


#### 3 Cross-Architecture Generalization

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, norm='instancenorm'):
        super(BasicBlock, self).__init__()
        self.norm = norm
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.GroupNorm(planes, planes, affine=True) if self.norm == 'instancenorm' else nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.GroupNorm(planes, planes, affine=True) if self.norm == 'instancenorm' else nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.GroupNorm(self.expansion*planes, self.expansion*planes, affine=True) if self.norm == 'instancenorm' else nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, channel=3, num_classes=10, norm='instancenorm'):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.norm = norm

        self.conv1 = nn.Conv2d(channel, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.GroupNorm(64, 64, affine=True) if self.norm == 'instancenorm' else nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.classifier = nn.Linear(512 * 7 * 7, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, self.norm))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def embed(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        return out

def ResNet18(channel, num_classes):
    return ResNet(BasicBlock, [2,2,2,2], channel=channel, num_classes=num_classes)

In [ ]:
random_resnet = ResNet18(channel=3, num_classes=2)
random_optimizer = SGD(random_resnet.parameters(), lr=0.01, momentum=0.9)
random_resnet.to(device)

gaussian_resnet = ResNet18(channel=3, num_classes=2)
gaussian_optimizer = SGD(gaussian_resnet.parameters(), lr=0.01, momentum=0.9)
gaussian_resnet.to(device)

random_scheduler = CosineAnnealingLR(random_optimizer, T_max=20)
gaussian_scheduler = CosineAnnealingLR(gaussian_optimizer, T_max=20)

random_criterion = torch.nn.CrossEntropyLoss()
gaussian_criterion = torch.nn.CrossEntropyLoss()

In [ ]:
img_syn_loaded = torch.load('mhist_result/Random_synthetic_dataset.pt')
img_syn = img_syn_loaded['images']
labels_syn = img_syn_loaded['labels']

random_synthetic_dataset = TensorDataset(img_syn, labels_syn)

random_train_loader = DataLoader(random_synthetic_dataset, batch_size=32, shuffle=True)

training_model(random_resnet, random_optimizer, random_scheduler,
               random_criterion, random_train_loader)

<ipython-input-40-0b359c411be8>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  img_syn_loaded = torch.load('mhist_result/Random_synthetic_dataset.pt')


Epoch [1/20], Loss: 32.268003692626955
Epoch [2/20], Loss: 39.44756996154785
Epoch [3/20], Loss: 29.212764282226562
Epoch [4/20], Loss: 4.685376424789428
Epoch [5/20], Loss: 2.6848295879364015
Epoch [6/20], Loss: 6.472138633728028
Epoch [7/20], Loss: 7.4405185413360595
Epoch [8/20], Loss: 7.809136123657226
Epoch [9/20], Loss: 5.654773998260498
Epoch [10/20], Loss: 2.7839413022994997
Epoch [11/20], Loss: 1.3441771459579468
Epoch [12/20], Loss: 1.0177720069885254
Epoch [13/20], Loss: 0.7976915907859802
Epoch [14/20], Loss: 0.6735869002342224
Epoch [15/20], Loss: 0.6593744111061096
Epoch [16/20], Loss: 0.659995150566101
Epoch [17/20], Loss: 0.6528219270706177
Epoch [18/20], Loss: 0.6485850191116334
Epoch [19/20], Loss: 0.6478282546997071
Epoch [20/20], Loss: 0.6469524288177491


In [ ]:
img_syn_loaded = torch.load('mhist_result/Gaussian_synthetic_dataset.pt')
img_syn = img_syn_loaded['images']
labels_syn = img_syn_loaded['labels']

gaussian_synthetic_dataset = TensorDataset(img_syn, labels_syn)

gaussian_train_loader = DataLoader(gaussian_synthetic_dataset, batch_size=32, shuffle=True)

training_model(gaussian_resnet, gaussian_optimizer, gaussian_scheduler,
               gaussian_criterion, gaussian_train_loader)

<ipython-input-41-6edec6420be9>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  img_syn_loaded = torch.load('mhist_result/Gaussian_synthetic_dataset.pt')


Epoch [1/20], Loss: 27.499790954589844
Epoch [2/20], Loss: 48.69056098937988
Epoch [3/20], Loss: 23.277333908081054
Epoch [4/20], Loss: 7.074451627731324
Epoch [5/20], Loss: 18.41257499694824
Epoch [6/20], Loss: 8.774026336669921
Epoch [7/20], Loss: 14.243234460353852
Epoch [8/20], Loss: 9.628783111572266
Epoch [9/20], Loss: 8.684937906265258
Epoch [10/20], Loss: 3.9883569192886354
Epoch [11/20], Loss: 4.03148624420166
Epoch [12/20], Loss: 2.4375954669713975
Epoch [13/20], Loss: 1.3857206243276596
Epoch [14/20], Loss: 1.0457713508605957
Epoch [15/20], Loss: 1.7373177814483642
Epoch [16/20], Loss: 4.088797825574875
Epoch [17/20], Loss: 2.754590082168579
Epoch [18/20], Loss: 1.7311811256408691
Epoch [19/20], Loss: 1.7143951177597045
Epoch [20/20], Loss: 0.7049971771240234


In [ ]:
test_folder = 'mhist_dataset/test'

transform = transforms.Compose([
    transforms.ToTensor()
])

test_dataset = datasets.ImageFolder(root=test_folder, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
evaluate_model(random_resnet, test_loader)

Test Accuracy: 51.89%


In [ ]:
evaluate_model(gaussian_resnet, test_loader)

Test Accuracy: 61.21%
